In [40]:
import warnings
warnings.filterwarnings("ignore", category=DeprecationWarning)
import math
import urllib
import os
import os.path as osp
import nbformat
import json
import ssl
import sys
import time
import hashlib
import pickle
import io
import random
import getpass
import inspect
import socket
import tempfile
from datetime import datetime, date
from datetime import timedelta

import matplotlib
import pylab # for side effects on matplotlib
import pandas as pd
import numpy as np
import IPython.display as ipd


In [41]:
#functions
def array_to_html(arr, row_names=None, col_names=None, title=None):
    ''' array_to_html, if placed at the end of a Anaconda cell, will
        display the array in html.   It uses pandas.
    '''
    if len(arr.shape) not in [1, 2]:
        print('The array must be a one- or two-dimensional array. This array is',
             len(arr.shape), 'dimensions')
        return

    if title:
        ipd.display(ipd.HTML('<h1>' + title + '</h1>'))
    df = pd.DataFrame(arr, index=row_names, columns=col_names)
    return ipd.display(ipd.HTML(df.to_html()))
def separate(string):
    separated=[]
    semic=[-1]
    reverse=string[::-1]
    lastword=string[-reverse.find(';'):-1]
    for posi in range(len(string)):
        if string[posi]==';':
            semic.append(posi)
    for positionx in range(len(semic)-1):
        word=string[semic[positionx]+1:semic[positionx+1]]
        separated.append(float(word))
    separated.append(float(lastword))
    return separated
def notuntillfloat(name):
    a = input(name)
    try:
        b= float(a)
    except ValueError:
        print('Please enter a float')
        a= float(notuntillfloat(name))
    return a
#notuntillfloat('Please enter the Target OD 600 ')
def notuntillint(name):
    a = input(name)
    try:
        b= int(a)
    except ValueError:
        print('Please enter an integer')
        a= int(notuntillint(name))
    return a
#notuntillint('Please enter the total number of samples you have ')
def notuntillindex(name,array):
    a = int(notuntillint(name))
    try:
        b= array[a-1]
    except IndexError:
        print('The index you entered is out of bound. Please enter the currect position')
        a= int(notuntillindex(name,array))
    return a
#notuntillindex('Please enter the number of samples you want to change ',[1,2,3,4,5])

In [42]:
## diluting culture
print('The Diluter V4.2.1---Created by Harry Wang')
print('If the program does not work, please restart it from the first cell.')
print('You can directly copy the result to Ecxel')
print('If you have enough confidence, please use The Diluter Advanced. Stop this program first.')
maxresonableod=5
targetod=float(notuntillfloat('Please enter the Target OD 600: '))
if targetod>maxresonableod:
    print('This OD is too large. Please check if you enter the wrong number')
    targetod=float(notuntillfloat('Please reenter the Target OD 600: '))
numofhours=float(notuntillfloat('Please enter the number of hours you will grow the culture (hours): '))
desiredvol=float(notuntillfloat('Please enter the desired culture volume (ml): '))
doublehr=float(notuntillfloat('Please enter the doubling time of your strain (hours): '))
totalsample=int(notuntillint('Please enter the total number of samples you have: '))
samplesize=float(notuntillfloat('Please enter the volume (mL) of samples you have: '))
datatable=[]
conditions=np.array([targetod,numofhours,desiredvol,doublehr,totalsample,samplesize])
conditions=np.reshape(conditions,[1,6])
array_to_html(conditions,row_names=['Conditions'],col_names=['Target OD 600','Number of hours for growth','Desired culture volume','Doubling time','Number of samples','Sample volume'],title='Please Check The Conditions for Cultivation')
print('If you need to change the conditions, please restart the program')
for i in range(totalsample):
    d=i+1
    currentod=float(notuntillfloat('Please enter the Current OD 600 for sample %d: '%(d)))
    if currentod>maxresonableod:
        print('This OD is too large. Please check if you enter the wrong number')
        currentod=float(notuntillfloat('Please reenter the Current OD 600 for sample %d: '%(d)))
    amt=(desiredvol*(targetod/currentod))/(2**(numofhours/doublehr))*1000
    if (amt/1000)>samplesize:
        suggesthr=doublehr*math.log(desiredvol*(targetod/currentod)*1000/samplesize, 2)
        print('You do not have enough sample. Please restart the program and choose a larger number of hours ')
        print('The minimum hours you need is :',suggesthr)
    datatable.append([currentod,amt])
datatable=np.array(datatable)
print('The datatable table entered: ')
array_to_html(datatable, row_names=range(1,len(datatable)+1),col_names=['Current OD','Amount need to add (ul)'], title='The Data Entered')
#for item in datatable:
    #print(item)
change=input('Do you need to change any data? Please enter  \'y\' or \'n\': ')
while change!='n':
    num=int(notuntillint('Please enter the number of samples you want to change: '))
    for j in range(num):
        position = int(notuntillindex('Please enter the position of the sample you want to change: ',datatable))
        print('The value you want to change is: '+'Sample '+ str(position)+': OD '+str(datatable[position-1][0]))
        newdata=float(notuntillfloat('Please enter the correct OD: '))
        if newdata>maxresonableod:
            print('This OD is too large. Please check if you enter the wrong number')
            newdata=float(notuntillfloat('Please reenter the correct OD: '))
        newvol=(desiredvol*(targetod/newdata))/(2**(numofhours/doublehr))*1000
        datatable[position-1,0]=newdata
        datatable[position-1,1]=newvol
        print('The changed result is: '+'Sample '+ str(position)+': New OD '+str(datatable[position-1][0]))
    array_to_html(datatable, row_names=range(1,len(datatable)+1),col_names=['Current OD','Amount need to add (ul)'], title='The new result')
    change=input('Do you need to change any data? Please enter  \'y\' or \'n\': ')
array_to_html(datatable, row_names=range(1,len(datatable)+1),col_names=['Current OD','Amount need to add (ul)'], title='The Final result')

The Diluter V4.2.1---Created by Harry Wang
If the program does not work, please restart it from the first cell.
You can directly copy the result to Ecxel
If you have enough confidence, please use The Diluter Advanced. Stop this program first.
Please enter the Target OD 600: 0.6
Please enter the number of hours you will grow the culture (hours): 16
Please enter the desired culture volume (ml): 5
Please enter the doubling time of your strain (hours): 2
Please enter the total number of samples you have: 5
Please enter the volume (mL) of samples you have: 4


,Target OD 600,Number of hours for growth,Desired culture volume,Doubling time,Number of samples,Sample volume
Conditions,0.6,16.0,5.0,2.0,5.0,4.0


If you need to change the conditions, please restart the program
Please enter the Current OD 600 for sample 1: 3
Please enter the Current OD 600 for sample 2: 2.1
Please enter the Current OD 600 for sample 3: 1.3
Please enter the Current OD 600 for sample 4: 1.7
Please enter the Current OD 600 for sample 5: 0.69
The datatable table entered: 


,Current OD,Amount need to add (ul)
1,3.00,3.906250
2,2.10,5.580357
3,1.30,9.014423
4,1.70,6.893382
5,0.69,16.983696


Do you need to change any data? Please enter  'y' or 'n': y
Please enter the number of samples you want to change: 5
Please enter the position of the sample you want to change: 2.4
Please enter an integer
Please enter the position of the sample you want to change: n
Please enter an integer
Please enter the position of the sample you want to change: 1
The value you want to change is: Sample 1: OD 3.0
Please enter the correct OD: 3
The changed result is: Sample 1: New OD 3.0
Please enter the position of the sample you want to change: 1
The value you want to change is: Sample 1: OD 3.0
Please enter the correct OD: 4
The changed result is: Sample 1: New OD 4.0
Please enter the position of the sample you want to change: 1
The value you want to change is: Sample 1: OD 4.0
Please enter the correct OD: 3
The changed result is: Sample 1: New OD 3.0
Please enter the position of the sample you want to change: 1
The value you want to change is: Sample 1: OD 3.0
Please enter the correct OD: 3
The c

,Current OD,Amount need to add (ul)
1,3.00,3.906250
2,2.10,5.580357
3,1.30,9.014423
4,1.70,6.893382
5,0.69,16.983696


Do you need to change any data? Please enter  'y' or 'n': n


,Current OD,Amount need to add (ul)
1,3.00,3.906250
2,2.10,5.580357
3,1.30,9.014423
4,1.70,6.893382
5,0.69,16.983696


In [43]:
# diluting culture advanced
print('The Diluter Advanced V1.0.5---Created by Harry Wang')
print('If the program does not work, please restart it.')
print('You can directly copy the result to Ecxel')
maxresonableod=5
targetod=float(notuntillfloat('Please enter the Target OD 600: '))
if targetod>maxresonableod:
    print('This OD is too large. Please check if you enter the wrong number')
    targetod=float(notuntillfloat('Please reenter the Target OD 600: '))
numofhours=float(notuntillfloat('Please enter the number of hours you will grow the culture (hours): '))
desiredvol=float(notuntillfloat('Please enter the desired culture volume (ml): '))
doublehr=float(notuntillfloat('Please enter the doubling time of your strain (hours): '))
datatable=[]
conditions=np.array([targetod,numofhours,desiredvol,doublehr])
conditions=np.reshape(conditions,[1,4])
array_to_html(conditions,row_names=['Conditions'],col_names=['Target OD 600','Number of hours for growth','Desired culture volume','Doubling time'],title='Please Check The Conditions for Cultivation')
print('If you need to change the conditions, please restart the program')
total=input('Please enter the set of OD of all your samples to decimals separated with \' ; \' ')
print('The datatable you entered for future reference: ')
print(total)
#totalinlist=separate(total)
totalsample=np.array(total.split(';'))
for i in range(len(totalsample)):
    currentod=float(totalsample[i])
    d=i+1
    amt=(desiredvol*(targetod/currentod))/(2**(numofhours/doublehr))*1000
    datatable.append([totalsample[i],amt])
datatable=np.array(datatable)

print('The datatable table entered: ')
array_to_html(datatable, row_names=range(1,len(datatable)+1),col_names=['Current OD','Amount need to add (ul)'], title='The Data Entered')
#for item in datatable:
    #print(item)
change=input('Do you need to change any data? Please enter  \'y\' or \'n\' ')
while change!='n':
    num=int(notuntillint('Please enter the number of samples you want to change: '))
    for j in range(num):
        position = int(notuntillindex('Please enter the position of the sample you want to change: ',datatable))
        print('The value you want to change is: '+'Sample '+ str(position)+': OD '+str(datatable[position-1][0]))
        newdata=float(notuntillfloat('Please enter the correct OD: '))
        if newdata>maxresonableod:
            print('This OD is too large. Please check if you enter the wrong number')
            newdata=float(notuntillfloat('Please reenter the correct OD: '))
        newvol=(desiredvol*(targetod/newdata))/(2**(numofhours/doublehr))*1000
        datatable[position-1,0]=newdata
        datatable[position-1,1]=newvol
        print('The changed result is: '+'Sample '+ str(position)+': New OD '+str(datatable[position-1][0]))
    array_to_html(datatable, row_names=range(1,len(datatable)+1),col_names=['Current OD','Amount need to add (ul)'], title='The new result')
    change=input('Do you need to change any data? Please enter  \'y\' or \'n\' ')
array_to_html(datatable, row_names=range(1,len(datatable)+1),col_names=['Current OD','Amount need to add (ul)'], title='The Final result')

The Diluter Advanced V1.0.5---Created by Harry Wang
If the program does not work, please restart it.
You can directly copy the result to Ecxel
Please enter the Target OD 600: 0.6
Please enter the number of hours you will grow the culture (hours): 16
Please enter the desired culture volume (ml): 5
Please enter the doubling time of your strain (hours): 2


,Target OD 600,Number of hours for growth,Desired culture volume,Doubling time
Conditions,0.6,16.0,5.0,2.0


If you need to change the conditions, please restart the program
Please enter the set of OD of all your samples to decimals separated with ' ; ' 3;2.1;1.3;1.7;0.69
The datatable you entered for future reference: 
3;2.1;1.3;1.7;0.69
The datatable table entered: 


,Current OD,Amount need to add (ul)
1,3,3.9062499999999996
2,2.1,5.580357142857142
3,1.3,9.014423076923077
4,1.7,6.8933823529411775
5,0.69,16.983695652173914


Do you need to change any data? Please enter  'y' or 'n' n


,Current OD,Amount need to add (ul)
1,3,3.9062499999999996
2,2.1,5.580357142857142
3,1.3,9.014423076923077
4,1.7,6.8933823529411775
5,0.69,16.983695652173914
